# Case Study

As part of our journey to become the leader of the Homecare industry we want to be able to know all the relevant details around French homes in order to: 

- better understand the risk we insure and price our products accordingly
- anticipate the customers' needs and offer useful services

For that we are constantly looking for new data to analyze and enhance our product and services.

Taking into consideration the questions we already ask during the subscription process on our website:

- list 3 new data points which we don't have yet (example: the building year)

- code a scraper which would retrieve this data, clean it and store in a data model / db you propose

- explain (and even better code) how you would get prices to analyze the previous data (ex: getting prices with this extra data and analyze the relationship with the price)

# List 3 new data points which we don't have yet (example: the building year)

1.) Average annual home insurance cost for the area

2.) Cost of rebuilding homes in the area

3.) Hospitalization price per day for health insurance

4.) Average home insurance claims cost for the area

5.) Proximity of the home to a fire station (Caserne de pompiers)

6.) The house build(type of building materials)

## Explanation of the data points

### Average annual home insurance cost for the area
    Get the average annual home insurance cost for the area to compare if you are over pricing or under pricing. 

**Data to be scarped** 
   
    Average annual home insurance costs per area code(_code postale_) in France to be use in price comparison

### Hospitalization price per day for health insurance
    Get the price per day of hospitalization for health insurance so as to aggregate the cost of civil liability(_responsabilité civile_) included in the home insurance
    
 **Data to be scarped** 
   
    Daily hospitalization cost for Public and private hospital/clinic

### Cost of rebuilding homes in the area
    Get the construction index cost(ICC) and the FFB index of the trimester, get the price per m<sup>2</sup> for the home area to calculate the cost of the home(_using size of the home gotten during the subscription process on luko website_).
    
   **Data to be scarped** 
   
    FFB and INSEE ICC index
    Price per m2 per area(Town)

### Average claims cost for the area
    Get the average home insurance claim cost for the area. Areas with higher averages will have higher prices than those with lower prices.
    
   **Data to be scarped** 
   
    Get all average home insurance claim costs per area code(_code postale_) in France to be use in a pricing mix
    
### Proximity of the home to a fire station (_Caserne de pompiers_)
    Homes which are closer to fire stations will have lower prices than those which are far-off from a fire station
    
   **Data to be scarped** 
   
    Get the addresses of all fire stations in France, and for each new home to be insured, calculate the distance between the home and the fire station and set an index for this parameter 
      

# Code a scraper which would retrieve this data, clean it and store in a data model / db you propose

**Scraping the various data points**

In [1]:
# Loading the necessary packages

from requests_html import HTMLSession
import pandas as pd 
import requests

## Average annual home insurance cost for the area

In [2]:
annual_home_insurance_cost = "https://www.lesfurets.com/assurance-habitation/pres-de-chez-vous"

In [3]:
try:
    # establish/open a session
    session = HTMLSession()

    # submitting a GET request
    r = session.get(annual_home_insurance_cost)
    print(r.status_code)

except requests.exceptions.RequestException as e:
    print(e)

200


In [4]:
# r.content # Examine the page content 
# len(table_ann_av_cost)

table_ann_av_cost = pd.read_html(r.content) # Get all the tables in the page content
ann_av_cost_big_cities = table_ann_av_cost[1] # Slice out the annual averag cost for big cities
ann_av_cost_regions = table_ann_av_cost[2] # Slice out the annual averag cost for regions
ann_av_cost_big_cities.head()

,Ville,Prix moyen appartement,Prix moyen maison
0,Paris,257 €,384 €
1,Marseille,271 €,477 €
2,Lyon,224 €,403 €
3,Toulouse,214 €,381 €
4,Nice,275 €,561 €


In [5]:
ann_av_cost_regions.head()

,Région,Prix moyen appartement,Prix moyen maison
0,Alsace-Champagne-Ardenne-Lorraine,189 €,386 €
1,Aquitaine-Limousin-Poitou-Charentes,185 €,350 €
2,Auvergne-Rhône-Alpes,204 €,367 €
3,Bourgogne-Franche-Comté,185 €,366 €
4,Bretagne,162 €,303 €


### Cleaning the scraped data

**Removing the € sign from the data to ease future calculations**

In [6]:
ann_av_cost_big_cities["Prix moyen appartement"] = [float(i[:-1])/1.00  for i in ann_av_cost_big_cities["Prix moyen appartement"]]
ann_av_cost_big_cities["Prix moyen maison"] = [float(j[:-1])/1.00 for j in ann_av_cost_big_cities["Prix moyen maison"]]
ann_av_cost_big_cities.head()

,Ville,Prix moyen appartement,Prix moyen maison
0,Paris,257.0,384.0
1,Marseille,271.0,477.0
2,Lyon,224.0,403.0
3,Toulouse,214.0,381.0
4,Nice,275.0,561.0


In [7]:
ann_av_cost_regions["Prix moyen appartement"] = [float(i[:-1])/1.00 for i in ann_av_cost_regions["Prix moyen appartement"]]
ann_av_cost_regions["Prix moyen maison"] = [float(j[:-1])/1.00 for j in ann_av_cost_regions["Prix moyen maison"]]
ann_av_cost_regions.head()

,Région,Prix moyen appartement,Prix moyen maison
0,Alsace-Champagne-Ardenne-Lorraine,189.0,386.0
1,Aquitaine-Limousin-Poitou-Charentes,185.0,350.0
2,Auvergne-Rhône-Alpes,204.0,367.0
3,Bourgogne-Franche-Comté,185.0,366.0
4,Bretagne,162.0,303.0


In [8]:
ann_av_cost_regions["Région"][12] = "Provence-Alpes-Côte d`Azur"
# ann_av_cost_regions

<ipython-input-8-40797e98fa24>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ann_av_cost_regions["Région"][12] = "Provence-Alpes-Côte d`Azur"


## Hospitalization price per day for health insurance

In [9]:
daily_hsp_cost = "https://www.aide-sociale.fr/prix-hospitalisation/"

In [10]:
try:
    # establish/open a session
    session = HTMLSession()

    # submitting a GET request
    r_daily_hsp_cost = session.get(daily_hsp_cost)
    print(r_daily_hsp_cost.status_code)

except requests.exceptions.RequestException as e:
    print(e)

200


### Cleaning the scraped data

In [11]:
# r_daily_hsp_cost.html.find('.list-1')[2].find('li')[0].text[:8]

# for i in r_daily_hsp_cost.html.find('.list-1')[2].find('li'):
#     print(i.text[:8])



daily_hos_rates = [i.text[:8] for i in r_daily_hsp_cost.html.find('.list-1')[2].find('li')] # Get a list of the daily rates
# ['20 euros','15 euros']

daily_hos_rates = [int(i.split()[0])/1.00 for i in daily_hos_rates]# Split and remove 'euros' from data to get only int

data = [["Clinic", "Phsy"],daily_hos_rates] # create a 'data' ndarray
cols = data.pop(0) # Remove redundant header(i.e the first row) from array
daily_hos_rates = pd.DataFrame(data, columns=cols) # Create a data frame
daily_hos_rates

,Clinic,Phsy
0,20.0,15.0


## Cost of rebuilding homes in the area

**INDICE FFB (_BÂTIMENT_)**

In [12]:
base_url_indice_FFB = "https://grouperouge.fr/indice_ffb_assurance_entreprise_118_4.html"

In [13]:
table_FFB = pd.read_html(base_url_indice_FFB) # pd.read_html uses Beautifulsoup in the background to extract the table
table_FFB[0].head()

,0,1,2,3,4
0,NaN,Au 31 mars,Au 30 juin,Au 30 septembre,Au 31 décembre
1,2020.0,995.10,995.20,996.80,1000.50
2,2019.0,993.50,994.50,994.20,994.30
3,2018.0,981.80,988.10,987.50,988.20
4,2017.0,955.80,960.10,965.60,974.80


#### Cleaning the scraped data

In [14]:
table_FFB[0].columns = 'Year','T1','T2','T3','T4' # Rename the columns
table_FFB[0] = table_FFB[0].drop([0], axis=0) # Delete the first row which has no meaning
table_FFB[0]["Year"] = [int(i) for i in table_FFB[0]["Year"]] # convert years to integer
table_FFB[0].reset_index(drop=True, inplace=True)
table_FFB[0].head()

,Year,T1,T2,T3,T4
0,2020,995.10,995.20,996.80,1000.50
1,2019,993.50,994.50,994.20,994.30
2,2018,981.80,988.10,987.50,988.20
3,2017,955.80,960.10,965.60,974.80
4,2016,929.50,931.20,935.90,942.00


**Indice du coût de la construction (_ICC_)**

In [15]:
base_url_indice_ICC = "https://www.labase-lextenso.fr/indice-du-cout-de-la-construction-icc-et-indice-de-reference-des-loyers-irl"
try:
    # establish/open a session
    session = HTMLSession()

    # submitting a GET request
    r_icc = session.get(base_url_indice_ICC)
    print(r_icc.status_code)
    table_icc_data = pd.read_html(r_icc.content)

except requests.exceptions.RequestException as e:
    print(e)

200


In [16]:
# len(table_icc_data)# Inspect to see if there are multple tables
table_icc_data[0]# Get the first table in the list

,0,1,2,3,4,5,6,7,8,9
0,ANNÉE,ICC1er trim.,IRL 1er trim.,ICC2e trim.,IRL2e trim.,ICC3e trim.,IRL3e trim.,ICC4e trim.,IRL4e trim.,NaN
1,ANNÉE,ICC1er trim.,IRL 1er trim.,ICC2e trim.,IRL2e trim.,ICC3e trim.,IRL3e trim.,ICC4e trim.,IRL4e trim.,NaN
2,2004,1225,10780,1267,10828,1272,10872,1269,10902,NaN
3,2005,1270,10964,1276,11008,1278,11057,1332,11101,NaN
4,2006,1362,11147,1366,11198,1381,11243,1406,11277,NaN
5,2007,1385,11307,1435,11337,1443,11368,1474,11430,NaN
6,2008,1497,11512,1562,11607,1594,11703,1523,11754,NaN
7,2009,1503,11770,1498,11759,1502,11741,1507,11747,NaN
8,2010,1508,11781,1517,11826,1520,11870,1533,11917,NaN
9,2011,1554,11969,1593,12031,1624,12095,1638,12168,NaN


#### Cleaning the scraped data

In [17]:
table_icc = table_icc_data[0].copy() 
table_icc.head()

,0,1,2,3,4,5,6,7,8,9
0,ANNÉE,ICC1er trim.,IRL 1er trim.,ICC2e trim.,IRL2e trim.,ICC3e trim.,IRL3e trim.,ICC4e trim.,IRL4e trim.,NaN
1,ANNÉE,ICC1er trim.,IRL 1er trim.,ICC2e trim.,IRL2e trim.,ICC3e trim.,IRL3e trim.,ICC4e trim.,IRL4e trim.,NaN
2,2004,1225,10780,1267,10828,1272,10872,1269,10902,NaN
3,2005,1270,10964,1276,11008,1278,11057,1332,11101,NaN
4,2006,1362,11147,1366,11198,1381,11243,1406,11277,NaN


In [18]:
icc_cols = [0,1,3,5,7] # Building a list of ICC columns to be use to sliced out from table_icc
table_icc = table_icc[icc_cols]
table_icc.head()

,0,1,3,5,7
0,ANNÉE,ICC1er trim.,ICC2e trim.,ICC3e trim.,ICC4e trim.
1,ANNÉE,ICC1er trim.,ICC2e trim.,ICC3e trim.,ICC4e trim.
2,2004,1225,1267,1272,1269
3,2005,1270,1276,1278,1332
4,2006,1362,1366,1381,1406


In [19]:
table_icc.columns = 'Year','T1','T2','T3','T4' # change the column names 
table_icc.head()

,Year,T1,T2,T3,T4
0,ANNÉE,ICC1er trim.,ICC2e trim.,ICC3e trim.,ICC4e trim.
1,ANNÉE,ICC1er trim.,ICC2e trim.,ICC3e trim.,ICC4e trim.
2,2004,1225,1267,1272,1269
3,2005,1270,1276,1278,1332
4,2006,1362,1366,1381,1406


In [20]:
table_icc = table_icc.drop(table_icc.tail(2).index) # delete the last two rows which contain Null values
table_icc = table_icc.drop([0,1]) # delete the first two rows which are redundant column values
table_icc.reset_index(drop=True, inplace=True)
table_icc.head()

,Year,T1,T2,T3,T4
0,2004,1225,1267,1272,1269
1,2005,1270,1276,1278,1332
2,2006,1362,1366,1381,1406
3,2007,1385,1435,1443,1474
4,2008,1497,1562,1594,1523


### Price per m2 per area(Town)

In [21]:
price_per_2 = "https://www.efficity.com/prix-immobilier-m2/"

In [22]:
try:
    # establish/open a session
    session = HTMLSession()

    # submitting a GET request
    r_price_per_2 = session.get(price_per_2)
    print(r_price_per_2.status_code)

except requests.exceptions.RequestException as e:
    print(e)

200


In [23]:
r_price_per_2_list = r_price_per_2.html.find('.squaremeter-locations-list-wrapper')[1].find('li')
# len(r_price_per_2_list)

# for i in r_price_per_2_list:
#     print(i.text.split())
    
price_data = [i.text.split() for i in r_price_per_2_list]   
price_data

[['Aix-en-Provence', '(13)', '4950€/m2'],
 ['Ajaccio', '(2A)', '4170€/m2'],
 ['Amiens', '(80)', '2200€/m2'],
 ['Angers', '(49)', '3190€/m2'],
 ['Avignon', '(84)', '2250€/m2'],
 ['Bastia', '(2B)', '1900€/m2'],
 ['Belfort', '(90)', '1310€/m2'],
 ['Besançon', '(25)', '2150€/m2'],
 ['Biguglia', '(2B)', '2670€/m2'],
 ['Blois', '(41)', '1650€/m2'],
 ['Bordeaux', '(33)', '4880€/m2'],
 ['Borgo', '(2B)', '2800€/m2'],
 ['Bourges', '(18)', '1470€/m2'],
 ['Brest', '(29)', '2020€/m2'],
 ['Béziers', '(34)', '1770€/m2'],
 ['Caen', '(14)', '2770€/m2'],
 ['Chalon-sur-Saône', '(71)', '1410€/m2'],
 ['Cherbourg-Octeville', '(50)', '2030€/m2'],
 ['Cholet', '(49)', '1900€/m2'],
 ['Châteauroux', '(36)', '1200€/m2'],
 ['Clermont-Ferrand', '(63)', '2360€/m2'],
 ['Dijon', '(21)', '2640€/m2'],
 ['Dunkerque', '(59)', '1790€/m2'],
 ['Grenoble', '(38)', '2570€/m2'],
 ['La', 'Rochelle', '(17)', '4380€/m2'],
 ['Le', 'Havre', '(76)', '2200€/m2'],
 ['Le', 'Mans', '(72)', '1790€/m2'],
 ['Lille', '(59)', '3630€/m2'],
 ['

In [24]:
# Get indexes of elements with length greater than 3

ct=0
gt4_indices = []
for i in price_data:
    if len(i) > 3:
       gt4_indices.append(ct)
    ct+=1   
gt4_indices

[24, 25, 26]

In [25]:
# Concatenate seperate name of towns with more than one word (e.g ['Le', 'Havre', '(76)', '2200€/m2'])

for k in gt4_indices:
    txt=""
    for i in price_data[k][:-2]:
        txt += str(i)+' '
    txt = [txt]
    # txt

    for i in price_data[k][-2:]:
        txt.append(i)
    price_data[k] = txt 
price_data

[['Aix-en-Provence', '(13)', '4950€/m2'],
 ['Ajaccio', '(2A)', '4170€/m2'],
 ['Amiens', '(80)', '2200€/m2'],
 ['Angers', '(49)', '3190€/m2'],
 ['Avignon', '(84)', '2250€/m2'],
 ['Bastia', '(2B)', '1900€/m2'],
 ['Belfort', '(90)', '1310€/m2'],
 ['Besançon', '(25)', '2150€/m2'],
 ['Biguglia', '(2B)', '2670€/m2'],
 ['Blois', '(41)', '1650€/m2'],
 ['Bordeaux', '(33)', '4880€/m2'],
 ['Borgo', '(2B)', '2800€/m2'],
 ['Bourges', '(18)', '1470€/m2'],
 ['Brest', '(29)', '2020€/m2'],
 ['Béziers', '(34)', '1770€/m2'],
 ['Caen', '(14)', '2770€/m2'],
 ['Chalon-sur-Saône', '(71)', '1410€/m2'],
 ['Cherbourg-Octeville', '(50)', '2030€/m2'],
 ['Cholet', '(49)', '1900€/m2'],
 ['Châteauroux', '(36)', '1200€/m2'],
 ['Clermont-Ferrand', '(63)', '2360€/m2'],
 ['Dijon', '(21)', '2640€/m2'],
 ['Dunkerque', '(59)', '1790€/m2'],
 ['Grenoble', '(38)', '2570€/m2'],
 ['La Rochelle ', '(17)', '4380€/m2'],
 ['Le Havre ', '(76)', '2200€/m2'],
 ['Le Mans ', '(72)', '1790€/m2'],
 ['Lille', '(59)', '3630€/m2'],
 ['Limoge

In [26]:
cols = ["Town", "Dept. Code", "Price m2"]
list_town_prices = pd.DataFrame(price_data,columns=cols)
list_town_prices.head()

,Town,Dept. Code,Price m2
0,Aix-en-Provence,(13),4950€/m2
1,Ajaccio,(2A),4170€/m2
2,Amiens,(80),2200€/m2
3,Angers,(49),3190€/m2
4,Avignon,(84),2250€/m2


**Removing '()' and €/m2 from Dept. Code & Price m2 to ease future references & calculations**

In [27]:
list_town_prices["Price m2"] = [float(i[:-4])/1.00 for i in list_town_prices["Price m2"]]
list_town_prices["Dept. Code"] = [j[1:-1] for j in list_town_prices["Dept. Code"]]
list_town_prices.head()

,Town,Dept. Code,Price m2
0,Aix-en-Provence,13,4950.0
1,Ajaccio,2A,4170.0
2,Amiens,80,2200.0
3,Angers,49,3190.0
4,Avignon,84,2250.0


## Average claims cost for the area

[Average cost for comprehensive house insurance claims in France between 2017 and 2019, by type of claim](https://www.statista.com/statistics/1121385/average-cost-comprehensive-house-insurance-claims-type-france/)

**I searched online but could not get this data point.**

The link above is the closest I could get on getting data on home insurance claims in France

## Proximity of the home to a fire station (_Caserne de pompiers_)

Could not get data on this data point 

# Saving the scraped data in a DB

**NB** I decided to save the scraped data at the end after the scraping and cleaning of the data so as to have a an easy flow for a user reading this notebook. 

Normally each data point should be saved after it has been scraped and cleaned.

I will be using PostgreSQL for this exercise

### Setting up DB Connection 

In [28]:
import psycopg2

In [29]:
param_dic = {
    "host"      : "localhost",
    "database"  : "luko_db",
    "user"      : "postgres",
    "password"  : "my_pwd"
}    

In [30]:
# Creating connection function
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
        print('Connected to database')
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    return conn

In [31]:
# Connecting to the database
conn = connect(param_dic)

Connecting to the PostgreSQL database...
Connected to database


In [32]:
# There are other methods of inserting like execute_values() and execute_mogrify() but single_insert works best for records less than 100

def single_insert(conn, insert_req):
    """ Execute a single INSERT request """
    cursor = conn.cursor()
    try:
        cursor.execute(insert_req)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

## Inserting Avg home insurance cost for big cities

In [33]:
for i in ann_av_cost_big_cities.index:
    town = ann_av_cost_big_cities["Ville"][i]
    apart = ann_av_cost_big_cities["Prix moyen appartement"][i]
    hse = ann_av_cost_big_cities["Prix moyen maison"][i]
    query = """
    INSERT into avghomeinsurancecosttown(town, apartment, house) values('%s',%s,%s);
    """ % (town, float(apart), hse)
    single_insert(conn, query)

## Inserting Avg home insurance cost for regions

In [34]:
for i in ann_av_cost_regions.index:
    regions = ann_av_cost_regions["Région"][i]
    apart = ann_av_cost_regions["Prix moyen appartement"][i]
    hse = ann_av_cost_regions["Prix moyen maison"][i]
    query = """
    INSERT into avghomeinsurancecostregions(region, apartment, house) values('%s',%s,%s);
    """ % (str(regions), float(apart), float(hse))
    single_insert(conn, query)      

## Inserting Hospitalization price per day for health insurance

In [35]:
for i in daily_hos_rates.index:
    query = """
    INSERT into hospitalization(clinic, phsy) values('%s',%s);
    """ % (float(daily_hos_rates['Clinic']), float(daily_hos_rates['Phsy']))
    single_insert(conn, query)    

## Inserting indice FFB

In [36]:
for i in table_FFB[0].index:
    year = table_FFB[0]["Year"][i]
    t1 = table_FFB[0]["T1"][i]
    t2 = table_FFB[0]["T2"][i]
    t3 = table_FFB[0]["T3"][i]
    t4 = table_FFB[0]["T4"][i]
    query = """
    INSERT into indiceffb(year, t1, t2, t3, t4) values(%s,%s,%s,%s,%s);
    """ % (int(year),float(t1), float(t2), float(t3), float(t4))
    single_insert(conn, query)    

## 4.3  Inserting indice ICC

In [37]:
for i in table_icc.index:
    year = table_icc["Year"][i]
    t1 = table_icc["T1"][i]
    t2 = table_icc["T2"][i]
    t3 = table_icc["T3"][i]
    t4 = table_icc["T4"][i]
    query = """
    INSERT into indiceicc(year, t1, t2, t3, t4) values(%s,%s,%s,%s,%s);
    """ % (int(year),float(t1), float(t2), float(t3), float(t4))
    single_insert(conn, query)    

## Inserting Price per m2 per area(Town)

In [38]:
# list_town_prices["Town", "Dept. Code", "Price m2"]

for i in list_town_prices.index:
    town = list_town_prices["Town"][i]
    dpt_code = list_town_prices["Dept. Code"][i]
    price_m2 = list_town_prices["Price m2"][i]
    query = """
    INSERT into pricem2(town, dptcode, price) values('%s','%s',%s);
    """ % (str(town),str(dpt_code), float(price_m2))
    single_insert(conn, query)
    
# Close the connection
conn.close()    

# Explain (and even better code) how you would get prices to analyze the previous data (ex: getting prices with this extra data and analyze the relationship with the price)

To get prices,
- I will factor in the extra data scraped in luko's product pricing mix to get new/optimal product price for a given user subscribing.

- I would then either subscribe to the API service of an Insurance price comparator such as [lesfurets](https://www.lesfurets.com/), [lelynx](https://www.lelynx.fr/) or code multiple multiple scrapers to scrape luko's competitors websites whilst using the given user's data as request params for the scrapers. 
    
    These scrapers would simulate a user subscribing for a home insurance quote on the competitors website with the same request data to get their prices.
    
- Finally I would then compare luko's pricing with that of the competitors to analyze the relationship with the prices    